# Create a table with SRA IDs, readstats QC data, and submitter metadata

**Below are the steps taken in this notebook:**
1. Installs, import Statements & Global Variable Definitions
2. Read in tables as dataframes
3. Merge dataframes
4. Write merged dataframe to data table and file


## Installs

In [1]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [2]:
%%capture
%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [3]:
from firecloud import fiss
import pandas as pd      
import os                 
import subprocess       
import re                 
import io
import gcsfs

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT
from terra_pandas import dataframe_to_table, table_to_dataframe

## Global Variable Declarations

In [64]:
# AnVIL_HPRC WorkspaceBucket
anvil_hprc_bucket       = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"

# table filenames (expected in the workspace bucket, see below)
submitter_metadata = 'WUSTL_HPRC_HiFi_Year3_metadata.tsv'
sra_metadata = 'metadata-12876517-processed-ok.tsv'

# submission id
submission_id = 'WUSTL_HPRC_HiFi_Year3'

# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_WRANGLING_WUSTL_HPRC_HiFi_Year3
Workspace storage bucket: gs://fc-a7e6ae6b-860b-4519-80a5-277aeb967124/


# Read In tables as dataframes

## Readstats table
Find the individual readstats output files in the table and concatenate their contents.
Add the hifi filename and sample name.

In [56]:
# readstats
readstats_df = table_to_dataframe("readstats", workspace=WORKSPACE, workspace_namespace=PROJECT)
qc_list = []

for index, row in readstats_df.iterrows():

        sample_readstats_fp = row['ReadStatsReport']
        sample_readstats_fn = os.path.basename(sample_readstats_fp)
        df = pd.read_csv(sample_readstats_fp, header=None, sep='\t')
        df = df[df[0]=='sample.fastq']
        # add sample and file name
        #df = df.append({0:'undef', 1:'sample', 2:row['sample']}, ignore_index=True)
        df = df.append({0:'undef', 1:'filename', 2:os.path.basename(row['hifi'])}, ignore_index=True)
        # remove all but the variables (total_bp, quartile_25 etc)
        df2 = df.drop(df.columns[[0, 1]], axis=1)
        # make rownames ('total_bp', 'quartile_25' etc)
        df2.index = df.iloc[:,1]
        # remove the now meaningless 'sample.fastq' filename
        df2 = df2.drop(index=['file'])
        qc_list.append(df2)

# merge
readstats_df = pd.concat(qc_list, axis=1).transpose()
readstats_df.head()

1,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,quartile_50,quartile_75,N25,N50,N75,filename
2,517275,10233445025,10.23,51,53341,19783,17853,19373,21259,18131,19724,21738,m64043_220728_173215-bc1018.5mc.hifi_reads.bam
2,1465069,32183433807,32.18,199,72755,21967,18553,20771,24326,19094,21721,25959,m64136_220715_182717-bc1018.5mc.hifi_reads.bam
2,2018757,38730219265,38.73,139,53377,19185,16486,17935,20910,16770,18510,22318,m64136_211114_164241-bc1002.5mc.hifi_reads.bam
2,1621355,34138087985,34.14,406,56238,21055,19061,20477,22470,19270,20815,22972,m64043_211210_180342-bc1021.5mc.hifi_reads.bam
2,1586624,33361982455,33.36,62,54098,21027,19052,20458,22436,19260,20789,22926,m64043_211212_045901-bc1021.5mc.hifi_reads.bam


## Submitter metadata table

In [57]:
meta_fp = os.path.join(bucket, submitter_metadata)
metadata_df = pd.read_csv(meta_fp, sep='\t')
metadata_df = metadata_df[['filename', 'sample_ID', 'instrument_model', 'shear_method', 'size_selection', 'ccs_algorithm', 
    'polymerase_version', 'seq_plate_chemistry_version', 'generator_facility', 'generator_contact']]
metadata_df.head()

,filename,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m64043_220728_173215-bc1018.5mc.hifi_reads.bam,HG00140,PacBio Sequel II,Megaruptor 1,SageELF,6.0.0,P2.2,C2,Washington University,tgraves@wustl.edu
1,m64136_220719_122056-bc1018.5mc.hifi_reads.bam,HG00140,PacBio Sequel II,Megaruptor 1,SageELF,6.0.0,P2.2,C2,Washington University,tgraves@wustl.edu
2,m64136_220717_152248-bc1018.5mc.hifi_reads.bam,HG00140,PacBio Sequel II,Megaruptor 1,SageELF,6.0.0,P2.2,C2,Washington University,tgraves@wustl.edu
3,m64136_220715_182717-bc1018.5mc.hifi_reads.bam,HG00140,PacBio Sequel II,Megaruptor 1,SageELF,6.0.0,P2.2,C2,Washington University,tgraves@wustl.edu
4,m64136_220708_205611-bc1016.5mc.hifi_reads.bam,HG00323,PacBio Sequel II,Megaruptor 1,SageELF,6.0.0,P2.2,C2,Washington University,tgraves@wustl.edu


## SRA table
Must download after finishing submission.
The downloaded table needs to be split in one row per file (instead of filename, filename2 <...> filenameN)

In [59]:
## Read in SRA file and split by filename

sra_file = os.path.join(bucket + sra_metadata)
sample_df = pd.read_csv(sra_file, sep='\t')

fnames = [x for x in sample_df.columns if x.startswith('filename')]
sra_df = pd.melt(sample_df, id_vars =['sample_name', 'accession'], value_vars = fnames, value_name='file')
sra_df= sra_df.dropna()
sra_df = sra_df[['file', 'accession', 'sample_name']]
sra_df.columns = ['filename', 'accession', 'sample']
sra_df.head()


,filename,accession,sample
0,m64043_220728_173215-bc1018.5mc.hifi_reads.bam,SRR23732295,HG00140
1,m64136_220708_205611-bc1016.5mc.hifi_reads.bam,SRR23732294,HG00323
2,m64136_211114_164241-bc1002.5mc.hifi_reads.bam,SRR23732283,HG00408
3,m64043_211210_180342-bc1021.5mc.hifi_reads.bam,SRR23732280,HG00597
4,m64043_211126_061537.5mc.hifi_reads.bam,SRR23732279,HG01192


# Merge all dataframes

In [60]:
print(sra_df.shape, readstats_df.shape, metadata_df.shape)

(79, 3) (79, 13) (79, 10)


In [61]:
merged_df = pd.merge(
    sra_df,
    readstats_df,
    on='filename')
merged_df = pd.merge(
    merged_df,
    metadata_df,
    on='filename')
merged_df.head()

,filename,accession,sample,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,...,N75,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m64043_220728_173215-bc1018.5mc.hifi_reads.bam,SRR23732295,HG00140,517275,10233445025,10.23,51,53341,19783,17853,...,21738,HG00140,PacBio Sequel II,Megaruptor 1,SageELF,6.0.0,P2.2,C2,Washington University,tgraves@wustl.edu
1,m64136_220708_205611-bc1016.5mc.hifi_reads.bam,SRR23732294,HG00323,1744844,33478490857,33.48,61,61839,19187,16250,...,22473,HG00323,PacBio Sequel II,Megaruptor 1,SageELF,6.0.0,P2.2,C2,Washington University,tgraves@wustl.edu
2,m64136_211114_164241-bc1002.5mc.hifi_reads.bam,SRR23732283,HG00408,2018757,38730219265,38.73,139,53377,19185,16486,...,22318,HG00408,PacBio Sequel II,Megaruptor 1,SageELF,6.0.0,P2.2,C2,Washington University,tgraves@wustl.edu
3,m64043_211210_180342-bc1021.5mc.hifi_reads.bam,SRR23732280,HG00597,1621355,34138087985,34.14,406,56238,21055,19061,...,22972,HG00597,PacBio Sequel II,Megaruptor 1,SageELF,6.0.0,P2.2,C2,Washington University,tgraves@wustl.edu
4,m64043_211126_061537.5mc.hifi_reads.bam,SRR23732279,HG01192,1830528,37677927562,37.68,153,53478,20583,18074,...,23304,HG01192,PacBio Sequel II,Megaruptor 1,SageELF,6.0.0,P2.2,C2,Washington University,tgraves@wustl.edu


# Upload To Tables

In [67]:
## Create table
#dataframe_to_table("post_sra_metadata", merged_df, WORKSPACE, PROJECT)
outf = os.path.join(bucket, submission_id + '_post_sra_metadata.tsv')
merged_df.to_csv(outf, sep="\t", index=False)